In [1]:
%run util.py

In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

# Loading JSON data
with open(data_folder + 'jam_ref_data.json', 'r') as json_file:
    jam_ref_data = json.load(json_file)
    
with open(data_folder + 'jam_test_data.json', 'r') as json_file:
    jam_test_data = json.load(json_file)

### Quantize the features by k-means clustering

In [4]:
### Quantize the features by k-means clustering 

import numpy as np

loc_list = []
length_list = []
numPts_list = []
speed_list = []

ref_loc_list = []
ref_length_list = []
ref_numPts_list = []
ref_speed_list = []
for key in jam_ref_data.keys():
    ref_loc_list.append(jam_ref_data[key]['longitude_latitude'])
    loc_list.append(jam_ref_data[key]['longitude_latitude'])
    
    ref_length_list.append(jam_ref_data[key]['length'])
    length_list.append(jam_ref_data[key]['length'])
    
    ref_numPts_list.append(float(jam_ref_data[key]['numPts']))
    numPts_list.append(float(jam_ref_data[key]['numPts']))
    
    ref_speed_list.append(jam_ref_data[key]['speed'])
    speed_list.append(jam_ref_data[key]['speed'])


test_loc_list = []
test_length_list = []
test_numPts_list = []
test_speed_list = []
for key in jam_test_data.keys():
    test_loc_list.append(jam_test_data[key]['longitude_latitude'])
    loc_list.append(jam_test_data[key]['longitude_latitude'])
    
    test_length_list.append(jam_test_data[key]['length'])
    length_list.append(jam_test_data[key]['length'])
    
    test_numPts_list.append(float(jam_test_data[key]['numPts']))
    numPts_list.append(float(jam_test_data[key]['numPts']))
    
    test_speed_list.append(jam_test_data[key]['speed'])
    speed_list.append(jam_test_data[key]['speed'])

loc_data = loc_list
length_data = np.array(length_list)
numPts_data = np.array(numPts_list)
speed_data = np.array(speed_list)

In [5]:
len(loc_data), len(ref_loc_list), len(test_loc_list)

(10808, 6926, 3882)

### Quantization levels: loc_k = 3, length_k = 2, numPts_k = 1, speed_k = 2

In [6]:
loc_data = [list(np.reshape(loc_data[i], np.size(loc_data[i]))) for i in range(len(loc_data))]

from scipy.cluster.vq import kmeans2
from ClusterAlg import DF, KMedians

loc_k = 3
num_core = 12
loc_label, loc_centroid, distToClusterCenter = KMedians(loc_data, loc_k, num_core, DF)

In [7]:
length_k = 2
length_centroid, length_label = kmeans2(length_data, length_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

numPts_k = 1
numPts_centroid, numPts_label = kmeans2(numPts_data, numPts_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

speed_k = 2
speed_centroid, speed_label = kmeans2(speed_data, speed_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

In [8]:
distToClusterCenter, loc_label, loc_centroid

([99.92522144693496,
  445.8037242410911,
  473.2524520380957,
  306.41220613893177,
  507.1285806381196,
  78.60391906034502,
  398.0016774419086,
  130.07462582630058,
  179.28007392726892,
  73.98203898493344,
  668.9159550390577,
  161.65663780060896,
  722.0061817597845,
  673.6750594871428,
  130.07462582630058,
  99.92522144693496,
  306.41220613893177,
  47.78461731234449,
  24.44591388152333,
  259.81047223173397,
  286.0378823731422,
  259.81047223173397,
  1231.6882085275747,
  5.642761065851067,
  137.7732315228792,
  715.4197282925949,
  808.1596340946932,
  1497.1345762053115,
  1963.9664837259727,
  1972.248525000684,
  1947.0585126572219,
  1041.497001826911,
  2598.348757024553,
  562.2693672096229,
  1214.6724856544452,
  91.22558456658152,
  387.4256421363353,
  6.0641819159206065,
  1288.0083803893929,
  180.8512686033382,
  142.71953016184852,
  642.1248978926517,
  298.7908009089836,
  434.045467320365,
  450.2770173879914,
  680.798365726891,
  673.6750594871428,

In [9]:
ref_loc_label = loc_label[0:np.size(np.array(ref_loc_list),0)]
test_loc_label = loc_label[np.size(np.array(ref_loc_list),0):np.size(loc_data,0)]

ref_length_label = length_label[0:np.size(np.array(ref_length_list),0)]
test_length_label = length_label[np.size(np.array(ref_length_list),0):np.size(length_data,0)]

ref_numPts_label = numPts_label[0:np.size(np.array(ref_numPts_list),0)]
test_numPts_label = numPts_label[np.size(np.array(ref_numPts_list),0):np.size(numPts_data,0)]

ref_speed_label = speed_label[0:np.size(np.array(ref_speed_list),0)]
test_speed_label = speed_label[np.size(np.array(ref_speed_list),0):np.size(speed_data,0)]

In [10]:
len(ref_loc_label), len(test_loc_label), len(ref_length_label), len(test_length_label), \
len(ref_numPts_label), len(test_numPts_label), len(ref_speed_label), len(test_speed_label)

(6926, 3882, 6926, 3882, 6926, 3882, 6926, 3882)

In [11]:
quantized_jam_ref_data = {}

i = 0
for key in jam_ref_data.keys():
    data = {'uuid': jam_ref_data[key]['uuid'],
            'longitude_latitude': jam_ref_data[key]['longitude_latitude'],
            'length': jam_ref_data[key]['length'],
            'numPts': float(jam_ref_data[key]['numPts']),
            'speed': jam_ref_data[key]['speed'],
            'startTime': jam_ref_data[key]['startTime'],
            'loc_label': float(ref_loc_label[i]),
            'length_label': float(ref_length_label[i]),
            'numPts_label': float(ref_numPts_label[i]),
            'speed_label': float(ref_speed_label[i])}
    quantized_jam_ref_data[key] = data
    i += 1
    
# Writing JSON data
with open(data_folder + 'quantized_jam_ref_data.json', 'w') as json_file:
    json.dump(quantized_jam_ref_data, json_file)

In [12]:
quantized_jam_test_data = {}

i = 0
for key in jam_test_data.keys():
    data = {'uuid': jam_test_data[key]['uuid'],
            'longitude_latitude': jam_test_data[key]['longitude_latitude'],
            'length': jam_test_data[key]['length'],
            'numPts': float(jam_test_data[key]['numPts']),
            'speed': jam_test_data[key]['speed'],
            'startTime': jam_test_data[key]['startTime'],
            'loc_label': float(test_loc_label[i]),
            'length_label': float(test_length_label[i]),
            'numPts_label': float(test_numPts_label[i]),
            'speed_label': float(test_speed_label[i])}
    quantized_jam_test_data[key] = data
    i += 1
    
# Writing JSON data
with open(data_folder + 'quantized_jam_test_data.json', 'w') as json_file:
    json.dump(quantized_jam_test_data, json_file)